<a href="https://colab.research.google.com/github/yunions/renewable/blob/master/data_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta, time
from tqdm import tqdm
# pandas max row 
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [ ]:
url = '/content/drive/MyDrive/KDT/'
feature_list = ['3시간기온', '강수형태','강수확률','습도', '풍속','풍향','하늘상태']
place = '감천제1동' #부산발전소 위치
period = ['201706_201801','201802_201812','201901_201912','202001_202012','202101_202106']

In [ ]:
#merge all csv & using only UTC 17 time
def call_all_csv(feature = '') :
  
  df = pd.read_csv(url+place+'_'+feature+'_'+period[0]+'.csv')
  df.columns = [' format: day', 'hour', 'forecast', 'value']
  #hour : predicting time / foracast : value

  for i in period[1:] :
    
    df01 = pd.read_csv(url+place+'_'+feature+'_'+i+'.csv')
    df01.columns = [' format: day', 'hour', 'forecast', 'value']
    #daytime is in header -> add data to column
    start_name = ' Start : ' + i[0:8]
    df=df.append({' format: day' : start_name} , ignore_index=True)
    df = df.append(df01, ignore_index = True)

  df = df.reset_index(drop=True)
  add_yymm_list(df)
  df = df.astype({'hour' : 'str'})
  df = df[df['hour'] == '1700.0'] #using UTC 17:00, KST 2:00

  df = add_forcasttime(df)
  df = df.drop({' format: day', 'hour', 'forecast','forecast_time', 'forcast_date'},1)

  return df    

In [ ]:
  def add_yymm_list(dataframe = '') :
    date = dataframe[dataframe['hour'].isnull() == True].copy() #date info row
    date['yymm'] = date[' format: day'].str.slice(start=9, stop=15)
    copy_df = dataframe[' format: day'].copy()
    #display(date)
    index = date.index

    x = 0

    for i in range(len(index)):
      y = index[i]

      if x == 0 :
        x = index[0]
        copy_df[:] = '201706'

      else : 
        date_info = date['yymm'][x]
        copy_df[x:y] = date_info
        x = index[i]
      
      if x == index[-1] :
        copy_df[x:] = '202106'

    dataframe['forecast_time'] = copy_df
    return dataframe[dataframe['hour'].isnull() == False]

In [ ]:
def add_forcasttime(dataframe = '') :
  copy_df = dataframe[dataframe['hour'].isnull() == False].copy()
  copy_df[' format: day'] = copy_df[' format: day'].apply(lambda x: "{:0>2d}".format(int(x)))
  copy_df['time'] = copy_df['forecast'] + 2
  copy_df['forcast_date'] = dataframe['forecast_time'] + copy_df[' format: day']
  copy_df['forcast_time'] = pd.to_datetime(copy_df['forcast_date'])

  return copy_df

#기상청데이터

## feature별 묶기
- Forecast time : 예보 발표 시점
- forecast : 예보 시간 (ex - Forecast time:2018-03-01 11:00:00, forecast:4.0 => 2018-03-01 11:00:00에 발표한 2018-03-01 15:00:00 예보
- 강수형태 : 없음 : 0, 비 : 1, 비/눈 : 2, 눈 : 3, 소나기 : 4
- 하늘상태 : 맑음 : 1, 구름많음 : 3, 흐림 : 4

In [ ]:
df_weather = pd.DataFrame()

for i in feature_list :
  tmp = call_all_csv(i)
  tmp = tmp[tmp['time'] < 25].copy()

  if i == '3시간기온' :
    df_weather['forcast_time'] = tmp['forcast_time'] + tmp['time'].apply(lambda x : timedelta(hours = x))

  df_weather[i] = tmp['value']

df_weather.head()

,forcast_time,3시간기온,강수형태,강수확률,습도,풍속,풍향,하늘상태
90,2017-06-01 06:00:00,18.0,0.0,0.0,80.0,2.1,108.0,1.0
91,2017-06-01 09:00:00,21.0,0.0,0.0,65.0,2.4,111.0,1.0
92,2017-06-01 12:00:00,22.0,0.0,0.0,65.0,2.2,114.0,1.0
93,2017-06-01 15:00:00,20.0,0.0,0.0,60.0,2.9,95.0,1.0
94,2017-06-01 18:00:00,19.0,0.0,0.0,65.0,1.0,86.0,1.0


# 부산태양광

In [ ]:
solar = pd.read_csv('/content/drive/MyDrive/KDT/solar_201706_202006.csv',encoding='cp949')
solar['년월일'] = solar['년월일'].str.replace('-','')
solar = solar.drop({ '호기', '총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', '최대(시간별, kw)',
       '최소(시간별, kw)'},1)

In [ ]:
solar.head()

,년월일,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,20170601,0,0,0,0,0,5.28,23.28,31.44,33.12,50.16,96.72,120.96,118.80,107.76,90.96,62.64,32.88,10.08,1.44,0.0,0,0,0,0
1,20170602,0,0,0,0,0,5.04,31.68,60.48,85.68,122.16,133.44,135.60,127.20,114.96,95.52,65.76,32.16,8.64,0.96,0.0,0,0,0,0
2,20170603,0,0,0,0,0,5.04,24.48,65.52,94.32,113.76,129.60,137.28,124.80,111.36,94.08,65.28,20.40,9.60,1.68,0.0,0,0,0,0
3,20170604,0,0,0,0,0,4.08,30.48,68.16,85.68,119.52,127.92,134.64,128.40,116.40,95.28,66.00,32.16,9.36,0.96,0.0,0,0,0,0
4,20170605,0,0,0,0,0,8.40,30.48,66.00,93.60,114.00,129.84,127.20,120.72,118.80,78.96,65.28,32.64,12.72,2.64,0.0,0,0,0,0


# 데이터종합

In [ ]:
start_date = datetime.datetime(2017, 6, 1, 0)
end_date = datetime.datetime(2020, 7, 1, 0)

In [ ]:
str_date_list = [] 
while start_date != end_date: 
  
  str_date_list.append(start_date.strftime("%Y%m%d%H")) 
  start_date += timedelta(hours=1)

In [ ]:
df_total = pd.DataFrame(index=str_date_list,dtype=float)

## solar

In [ ]:
solar_t = solar.transpose()	#행 열 전환
solar_t.rename(columns=solar_t.iloc[0], inplace=True)
solar_t = solar_t.drop(solar_t.index[0])

In [ ]:
solar_result = [0]

for i in solar_t.columns:
  for x in range(24) :
    solar_result.append(solar_t[i].values[x])

df_total['solar_result'] = solar_result[:-1]
df_total

,solar_result
2017060100,0.00
2017060101,0.00
2017060102,0.00
2017060103,0.00
2017060104,0.00
...,...
2020063019,2.64
2020063020,0.00
2020063021,0.00
2020063022,0.00


In [ ]:
df_total = df_total.reindex(df_total.columns.tolist() + feature_list, axis=1)
df_total

,solar_result,3시간기온,강수형태,강수확률,습도,풍속,풍향,하늘상태
2017060100,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060101,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060102,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060103,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060104,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2020063019,2.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020063020,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020063021,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020063022,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##weather

In [ ]:
for i in df_weather.index :
  tmp = df_weather.loc[i]
  datetime = tmp['forcast_time'].strftime("%Y%m%d%H")
  df_total.at[datetime,'3시간기온' : ] = tmp['3시간기온' :]

df_total

,solar_result,3시간기온,강수형태,강수확률,습도,풍속,풍향,하늘상태
2017060100,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060101,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060102,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060103,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060104,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2021063012,NaN,27.0,NaN,30.0,65.0,4.3,56.0,4.0
2021063015,NaN,27.0,NaN,20.0,65.0,4.5,63.0,3.0
2021063018,NaN,26.0,NaN,20.0,70.0,4.9,55.0,3.0
2021063021,NaN,23.0,NaN,0.0,80.0,3.9,42.0,1.0


In [ ]:
df_total_inter = df_total.interpolate()

In [ ]:
df_total_inter.head(30)

,solar_result,3시간기온,강수형태,강수확률,습도,풍속,풍향,하늘상태
2017060100,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060101,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060102,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060103,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060104,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060105,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060106,5.28,18.000000,0.0,0.000000,80.000000,2.100000,108.000000,1.000000
2017060107,23.28,19.000000,0.0,0.000000,75.000000,2.200000,109.000000,1.000000
2017060108,31.44,20.000000,0.0,0.000000,70.000000,2.300000,110.000000,1.000000
2017060109,33.12,21.000000,0.0,0.000000,65.000000,2.400000,111.000000,1.000000


# add altitude

In [ ]:
altitude = pd.read_csv('/content/drive/MyDrive/KDT/solar_info.csv')
altitude.info()

,altitudeMeridian,altitude_09,altitude_12,altitude_15,altitude_18,azimuth_09,azimuth_12,azimuth_15,azimuth_18,date
0,76.53,44.23,76.04,53.08,16.44,92.42,158.50,260.00,285.35,20170601
1,77.01,44.24,76.10,53.14,16.50,92.31,158.31,260.10,285.41,20170602
2,77.08,44.26,76.17,53.20,16.56,92.20,158.12,260.19,285.46,20170603
3,77.16,44.27,76.23,53.25,17.02,92.10,157.52,260.27,285.51,20170604
4,77.22,44.28,76.28,53.31,17.07,92.00,157.33,260.35,285.56,20170605


In [91]:
time_list = ['09','12','15','18']
df_alti = pd.DataFrame(columns=['date', 'altitude','azimuth'])

for i in time_list :
  tmp = altitude[['date','altitude_'+i,'azimuth_'+i]].copy()
  tmp['date'] = altitude['date'].astype('str') + i
  tmp.columns = ['date', 'altitude','azimuth']
  df_alti = df_alti.append(tmp, ignore_index=True)

df_alti

,date,altitude,azimuth
0,2017060109,44.23,92.42
1,2017060209,44.24,92.31
2,2017060309,44.26,92.20
3,2017060409,44.27,92.10
4,2017060509,44.28,92.00
...,...,...,...
5963,2021062718,18.23,286.00
5964,2021062818,18.24,285.56
5965,2021062918,18.25,285.52
5966,2021063018,18.26,285.47


In [92]:
df_total_inter['date'] = df_total_inter.index
df_total_inter

,solar_result,3시간기온,강수형태,강수확률,습도,풍속,풍향,하늘상태,date
2017060100,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017060100
2017060101,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017060101
2017060102,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017060102
2017060103,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017060103
2017060104,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017060104
...,...,...,...,...,...,...,...,...,...
2021063012,0.0,27.0,0.0,30.0,65.0,4.3,56.0,4.0,2021063012
2021063015,0.0,27.0,0.0,20.0,65.0,4.5,63.0,3.0,2021063015
2021063018,0.0,26.0,0.0,20.0,70.0,4.9,55.0,3.0,2021063018
2021063021,0.0,23.0,0.0,0.0,80.0,3.9,42.0,1.0,2021063021


In [93]:
df_total_inter['date'] = df_total_inter.index
total = pd.merge(df_total_inter, df_alti, on= 'date', how='outer', right_index=True)
total = total.drop('date',axis=1)
total.head(20)

,solar_result,3시간기온,강수형태,강수확률,습도,풍속,풍향,하늘상태,altitude,azimuth
2017060100,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060101,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060102,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060103,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060104,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060105,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060106,5.28,18.000000,0.0,0.0,80.000000,2.100000,108.000000,1.0,NaN,NaN
2017060107,23.28,19.000000,0.0,0.0,75.000000,2.200000,109.000000,1.0,NaN,NaN
2017060108,31.44,20.000000,0.0,0.0,70.000000,2.300000,110.000000,1.0,NaN,NaN
2017060109,33.12,21.000000,0.0,0.0,65.000000,2.400000,111.000000,1.0,44.23,92.42


In [94]:
total.to_csv('total_data.csv')